In [2]:
## import the packages

import numpy as np
import pandas as pd
import glob
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from geopy.geocoders import Nominatim
import pycountry_convert
import pickle
import time
from time import sleep
from numpy.random import uniform
from difflib import SequenceMatcher
import os
from geopy.geocoders import Nominatim
from geopy.geocoders import Bing
pd.options.display.max_rows = 50
pd.set_option('max_colwidth',400)

In [3]:
def sleep_r(length):
    if length == "s":
        return sleep(uniform(0.25, 0.5))
    elif length == "m":
        return sleep(uniform(1, 2))
    elif length == "l":
        return sleep(uniform(4, 6))
    else:
        print("error")

In [4]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
##Define your chromedriver settings

chrome_options = webdriver.ChromeOptions()
executable_path='/Users/kristof/chromedriver'

chrome_options.add_argument('--load-extension=' + '/Users/kristof/Desktop/1.14.18_0/')

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=executable_path)

/miniconda3/envs/myPy/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


## Following downloading the institutional profiles, this notebook helps assigning location data to institutions

### Let's compile first the list of institutions

In [6]:
##Define your directory, where the downloaded profiles are stored
path =''
allFiles = glob.glob(path + "/*.csv")


listinst=[]
for file_ in allFiles:
    #df = pd.read_csv(file_)
    institutions=str(file_).split('/')
    institution=institutions[-1]
    institution=institution.replace('.csv', '')
    #df['institution']=institution
    #df['institution']=df['institution'].str.replace('.csv', '')
    
    listinst.append(institution)

### We will use Wikipedia to enrich our publication dataset with additional information on institutions

#### Two things are important: location, and information if the instiution is a business unit

#### Location: usually the Wikipedia page contains geo data

#### Business: if the page contains a table with information on Industry/HQ/Products

#### First, these are checked in the code below 

In [43]:
def wiki_page():

    wiki_name=driver.find_element_by_id('firstHeading').text
    business=0
    sleep_r('l')
    
    try:
    
        geo=driver.find_element_by_class_name('geo-dec')
        geowiki=driver.execute_script("return arguments[0].innerHTML", geo)
        
    
    except:
        geowiki=np.nan
    
    
    try:
        table=driver.find_element_by_css_selector('table.infobox')
        try:
            industry=table.find_element_by_xpath(".//tbody/tr/*[contains(text(), 'Industry')]/..").text
        except:
            industry=np.nan
        try:
            hq=table.find_element_by_xpath(".//tbody/tr/*[contains(text(), 'Headquarters')]/..").text
        except:
            hq=np.nan
        try:
            products=table.find_element_by_xpath(".///tbody/tr/*[contains(text(), 'Products')]/..").text
        except:
            products=np.nan
        
        
        
        if industry is not np.nan:
            business=1
        if hq is not np.nan:
            business=1
        if products is not np.nan:
            business=1
        
                      
    except:
        business=0
        
        products=np.nan
        hq=np.nan
        industry=np.nan
        
        
      
    return wiki_name, business, products, hq, industry, geowiki



### Besides defining the searched elements, we also need to deal with a situation in which the Wikipedia search yields multiple results

#### We collect both the top result, and the result which is the most similar based on the Wikipage title to the institution's name

In [44]:
def search_page(inst):
    result=driver.find_element_by_class_name('searchresults')
    try:
        results=result.find_elements_by_class_name('mw-search-result')


        titlelist=[]
        linklist=[]
        simlist=[]

        for i in results:
            title=i.find_element_by_class_name('mw-search-result-heading').find_element_by_tag_name('a')
            title=title.text
            titlelist.append(title)


            link=i.find_element_by_class_name('mw-search-result-heading').find_element_by_tag_name('a').get_attribute('href')
            linklist.append(link)


        for title in titlelist:
            sim=similar(title, inst)
            simlist.append(sim)

        keys=['titles', 'links', 'similarities' ]
        values=[titlelist, linklist, simlist]
        wikidict=dict(zip(keys, values))

    
        wiki_df=pd.DataFrame.from_dict(wikidict)

        if len(wiki_df)>0:

            wiki_df_tops=wiki_df.iloc[0]
            wiki_df_mosts=wiki_df.loc[wiki_df['similarities'].idxmax()]

            sleep_r('l')
        else:
            wiki_df_tops=np.nan
            wiki_df_mosts=np.nan
            
    except:
        wiki_df_tops=np.nan
        wiki_df_mosts=np.nan
        
    
    return wiki_df_tops, wiki_df_mosts

### Now we can execute the script

In [52]:



n=1
listdf=[]
for inst in listtest:
    summarypage=False
    driver.get('https://en.wikipedia.org/w/index.php?search=' + '+'.join(inst.split(' ')) +
          '&title=Special%3ASearch&go=Go')
    
    sleep_r('l')
    
    
    
    try:
        searchpage=driver.find_element_by_id('mw-search-top-table')
        searchpage=True
    except:
        searchpage=False
    
    if searchpage is False:
        
        try:
            driver.find_element_by_id('disambigbox')
            print('crossroad page')
            summarypage=True
            business=np.nan
            wiki_name=driver.find_element_by_id('firstHeading').text
            business=np.nan
            products=np.nan
            hq=np.nan
            industry=np.nan
            geowiki=np.nan
        
        except:
            summarypage=False
        
        if summarypage is False:
            temp=wiki_page()
            wiki_name=temp[0]
            business=temp[1]
            products=temp[2]
            hq=temp[3]
            industry=temp[4]
            geowiki=temp[5]
        
        else:
            geowiki=np.nan
            
                    
    else: 
        temp=search_page(inst)
        wiki_df_top=temp[0]
        wiki_df_most=temp[1]
        if wiki_df_most is not np.nan:        
            driver.get(wiki_df_most['links'])
            
            #temp=wiki_page()
            temp=wiki_page()
            wiki_name=temp[0]
            business=temp[1]
            products=temp[2]
            hq=temp[3]
            industry=temp[4]
            geowiki=temp[5]
        
            
        
    
    print(n, inst, wiki_name, business, products, hq, industry, geowiki)
    keys=['institute', 'wiki_page', 'business', 'products', 'hq','industry', 'latlon']
    values=[[inst], [wiki_name], [business], [products], [hq], [industry], [geowiki]]
    final_dict=dict(zip(keys, values))
    final_df=pd.DataFrame.from_dict(final_dict)
    listdf.append(final_df)
    n=n+1


1 University of Washington, Seattle University of Washington 0 nan nan nan 47.655°N 122.308°W
2 Stony Brook University Stony Brook University 0 nan nan nan 40.915°N 73.124°W
3 University of Dundee University of Dundee 0 nan nan nan 56.45722°N 2.98028°W
4 Georgia Institute of Technology Georgia Institute of Technology 0 nan nan nan 33.77583°N 84.39472°W
5 University of Maryland, Baltimore University of Maryland, Baltimore 0 nan nan nan 39.29139°N 76.62500°W
6 Rochester Institute of Technology Rochester Institute of Technology 0 nan nan nan 43.084412°N 77.674949°W
7 University of Maryland University of Maryland, College Park 0 nan nan nan 38.98750°N 76.94000°W
8 IBM Thomas J. Watson Research Center Thomas J. Watson Research Center 0 nan nan nan 41.2102°N 73.803°W
9 Carnegie Mellon University Carnegie Mellon University 0 nan nan nan 40.443322°N 79.943583°W
10 University of Manchester University of Manchester 0 nan nan nan 53.46556°N 2.23361°W
11 3M USA 3M 1 nan Headquarters Maplewood, Min

### Here are our resuls

In [55]:
geowikidf=pd.concat(listdf)

In [54]:
geowikidf

,institute,wiki_page,business,products,hq,industry,latlon
0,"University of Washington, Seattle",University of Washington,0.0,NaN,NaN,NaN,47.655°N 122.308°W
0,Stony Brook University,Stony Brook University,0.0,NaN,NaN,NaN,40.915°N 73.124°W
0,University of Dundee,University of Dundee,0.0,NaN,NaN,NaN,56.45722°N 2.98028°W
0,Georgia Institute of Technology,Georgia Institute of Technology,0.0,NaN,NaN,NaN,33.77583°N 84.39472°W
0,"University of Maryland, Baltimore","University of Maryland, Baltimore",0.0,NaN,NaN,NaN,39.29139°N 76.62500°W
0,Rochester Institute of Technology,Rochester Institute of Technology,0.0,NaN,NaN,NaN,43.084412°N 77.674949°W
0,University of Maryland,"University of Maryland, College Park",0.0,NaN,NaN,NaN,38.98750°N 76.94000°W
0,IBM Thomas J. Watson Research Center,Thomas J. Watson Research Center,0.0,NaN,NaN,NaN,41.2102°N 73.803°W
0,Carnegie Mellon University,Carnegie Mellon University,0.0,NaN,NaN,NaN,40.443322°N 79.943583°W
0,University of Manchester,University of Manchester,0.0,NaN,NaN,NaN,53.46556°N 2.23361°W


### For the institutions, where the Wikipedia search was unsuccessful, the GeoPy package has been used

In [47]:
geopy_search=geowikidf[geowikidf['latlon'].isna()]['institute']

In [48]:
for search in geopy_search:
    print(search)

3M USA
Accenture Services Pvt Ltd., India
Amdahl Corporation
AT&T Inc.
AT&T Laboratories Florham Park


In [60]:
api_key='Ah4kFtBXPpY5uh0r61bayfO3XEq-WDMQDO9TpLk6YgbsgeyRTkNE6ersiAKo6A8w'
NoneType = type(None)

geolocator = Bing(api_key=api_key,user_agent="engineroom", timeout=5)

locationList=[]
instList=[]
latlonList=[]

keys=['location','institute','latlon_geopy']
values=[locationList,instList,latlonList]

for i in geopy_search:
    try:
        location=geolocator.geocode(i)
        if type(location) != NoneType:
            latlon_geopy=location[1]
            sleep(2)
        else:
            location=np.nan
            latlon=np.nan
    except AttributeError:
        print('AttributeError')
        location=np.nan
        latlon_geopy=np.nan
        
    instList.append(i) 
    locationList.append(location)
    latlonList.append(latlon_geopy)
    
    print(i)
    DictFromList=dict(zip(keys,values))
    locDf=pd.DataFrame.from_dict(DictFromList)    

3M USA
Accenture Services Pvt Ltd., India
Amdahl Corporation
AT&T Inc.
AT&T Laboratories Florham Park


In [57]:
locDf.head()

,location,institute,latlon_geopy
0,"(United States, (39.495914459228516, -98.98998260498047))",3M USA,"(39.495914459228516, -98.98998260498047)"
1,"(India, (22.493118286132812, 79.72701263427734))","Accenture Services Pvt Ltd., India","(22.493118286132812, 79.72701263427734)"
2,"(Corporation, Donegal, Ireland, (54.649078369140625, -8.43280029296875))",Amdahl Corporation,"(54.649078369140625, -8.43280029296875)"
3,"(Nashville, TN, United States, (36.1617546081543, -86.77731323242188))",AT&T Inc.,"(36.1617546081543, -86.77731323242188)"
4,"(Florham Park, NJ, United States, (40.784759521484375, -74.395751953125))",AT&T Laboratories Florham Park,"(40.784759521484375, -74.395751953125)"


In [50]:
geowikidf=pd.merge(geowikidf,locDf,on='institute',how='outer')

In [53]:
geowikidf['latlon_wiki_geopy']=np.where(geowikidf['latlon'].isna(),geowikidf['latlon_geopy'],geowikidf['latlon'])

In [56]:
geowikidf.head()

,Unnamed: 0,institute,wiki_page,business,products,hq,industry,latlon,location,latlon_geopy,latlon_wiki_geopy
0,0,"University of Washington, Seattle",University of Washington,0.0,NaN,NaN,NaN,47.655°N 122.308°W,NaN,NaN,47.655°N 122.308°W
1,0,Stony Brook University,Stony Brook University,0.0,NaN,NaN,NaN,40.915°N 73.124°W,NaN,NaN,40.915°N 73.124°W
2,0,University of Dundee,University of Dundee,0.0,NaN,NaN,NaN,56.45722°N 2.98028°W,NaN,NaN,56.45722°N 2.98028°W
3,0,Georgia Institute of Technology,Georgia Institute of Technology,0.0,NaN,NaN,NaN,33.77583°N 84.39472°W,NaN,NaN,33.77583°N 84.39472°W
4,0,"University of Maryland, Baltimore","University of Maryland, Baltimore",0.0,NaN,NaN,NaN,39.29139°N 76.62500°W,NaN,NaN,39.29139°N 76.62500°W


In [ ]:
geowikidf.to_csv('geowiki_geopy.csv')